In [1]:
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
le = LabelEncoder()

In [2]:
file = open("C:\\Users\\MGC\\PycharmProjects\\mlquestion8\\train\\train.csv")
n = sum(1 for line in file) - 1
s = 100000
skip = sorted(random.sample(range(1, n+1), n-s))
chunk = pd.read_csv("C:\\Users\\MGC\\PycharmProjects\\mlquestion8\\train\\train.csv", low_memory=False, chunksize=10000, skiprows=skip )
df = pd.concat(chunk)

In [3]:
df

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10011677979251422697,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15707,320,50,1722,0,35,-1,79
1,10119103183585110562,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15699,320,50,1722,0,35,-1,79
2,10174080741792148808,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15705,320,50,1722,0,35,-1,79
3,10249231601524841268,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,-1,79
4,10351744346171829832,0,14102100,1005,1,44637516,27e3c518,f028772b,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,958970513803490007,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,685d1c4c,2347f47a,...,1,2,24009,320,50,2749,0,43,100177,221
99996,9646680559114388348,0,14103023,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,1,2,22815,320,50,2647,2,39,100148,23
99997,9673581674400385741,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,03528b27,2347f47a,...,1,0,17016,320,50,1873,3,39,-1,23
99998,9856039725148471897,0,14103023,1005,1,5b4d2eda,16a36ef3,f028772b,ecad2386,7801e8d9,...,1,0,19771,320,50,2227,0,935,100079,48


In [4]:
# Dealing with outliers by capping

col = ['C15', 'C16', 'C19', 'C21']
for col in col:
    percentiles = df[col].quantile(0.98)
    if df[col].quantile(0.98) < 0.5 * df[col].max():
        df[col][df[col] >= percentiles] = percentiles
numerical = []
categorical = []

for col in df.columns:
    if df[col].dtype == "object":
        categorical.append(col)
    else:
        numerical.append(col)

In [6]:
for i in categorical:
    df[i] = le.fit_transform(df[i])
print(df.click.value_counts(normalize=True))
print("\n")
df.drop(['C20'], axis=1, inplace=True)
df.drop(['id', 'hour'], axis=1, inplace = True)
df.rename(columns={'click': 'y'}, inplace=True, errors='raise')


0    0.8287
1    0.1713
Name: click, dtype: float64




In [8]:
def model_eval(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    return accuracy, cnf_matrix

In [9]:
X = df.drop(['y'], axis=1)
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size= 0.3, random_state= 42)
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train, y_train)
y_preds = model_rf.predict(X_test)
accuracy, cnf_matrix = model_eval(model_rf, X_test, y_test)
print(accuracy)
print(cnf_matrix)

0.8035
[[23325  1536]
 [ 4359   780]]


In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_preds, y_test))

              precision    recall  f1-score   support

           0       0.94      0.84      0.89     27684
           1       0.15      0.34      0.21      2316

    accuracy                           0.80     30000
   macro avg       0.54      0.59      0.55     30000
weighted avg       0.88      0.80      0.84     30000

